In [1]:
import pandas as pd
import numpy as np
import json
import urllib
import time

In [2]:
xls = pd.ExcelFile('ArtistList_July2_withDetails.xlsx')

In [3]:
master_artists = pd.read_excel(xls, 'FinalArtistListwithHandles_Jul2')

In [4]:
master_artists.columns

Index(['Artist', 'Age', 'No. of active years', 'Pop', 'Rock', 'Hip Hop',
       'World Music', 'R&B', 'Electronic', 'Facebook', 'Instagram', 'Twitter',
       'Youtube', 'MusicBrainz'],
      dtype='object')

In [6]:
master_artists['MusicBrainz'].nunique()

141

In [7]:
df = pd.DataFrame(columns = ['artist_name','title','release_date','primary_type'])

In [8]:
for index, row in master_artists.iterrows():
    time.sleep(5)
    offset = 0
    while True:
        url = "http://musicbrainz.org/ws/2/release/?query=arid:" + row['MusicBrainz']+ "&fmt=json&inc=release-groups&limit=100&offset=" + str(offset)
        contents = urllib.request.urlopen(url).read()
        channels = json.loads(contents)
        offset += 100
        if len(channels['releases']) == 0:
            break
        for releases in channels['releases']:
            features = []
            features.append(row['Artist'])
            if 'title' in releases:
                features.append(releases['title'])
            else:
                features.append(np.NAN)

            if 'date' in releases:
                features.append(releases['date'])
            else:
                features.append(np.NAN)

            if 'release-group' in releases:
                if 'primary-type' in releases['release-group']:
                    features.append(releases['release-group']['primary-type'])
                else:
                    features.append(np.NAN)
            else:
                features.append(np.NAN)
            df = df.append(pd.Series(features,index = df.columns),ignore_index=True)

In [9]:
df.count()

artist_name     22720
title           22720
release_date    20077
primary_type    22556
dtype: int64

In [10]:
df['release_date'].isnull().values.sum()

2643

In [11]:
df = df.dropna(axis=0, subset=['release_date'])

In [12]:
df.count()

artist_name     20077
title           20077
release_date    20077
primary_type    19996
dtype: int64

In [13]:
df.dtypes

artist_name     object
title           object
release_date    object
primary_type    object
dtype: object

In [14]:
df['release_date'].str.len().unique()

array([10,  4,  0,  7], dtype=int64)

In [15]:
df = df[df['release_date'].str.len() != 0]

In [16]:
df.count()

artist_name     19491
title           19491
release_date    19491
primary_type    19412
dtype: int64

In [17]:
df[df['release_date'].str.len() == 4].release_date.describe()

count     5792
unique      57
top       2001
freq       256
Name: release_date, dtype: object

In [18]:
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d %H:%M:%S')

In [19]:
df.groupby(['artist_name','title']).count()

release_date  \
artist_name title                                                              
21 Savage   1Night                                                         1   
            Bartier Cardi                                                  2   
            Enzo                                                           1   
            Focus                                                          2   
            Free Guwop EP                                                  1   
            Gucci on My                                                    1   
            How To Ball                                                    1   
            Issa Album                                                     1   
            Krippy Kush (remix)                                            1   
            One Foot                                                       1   
            Red Opps                                                       1   
            Rover 2.0                                                      1   
            Savage Mode                                                    1   
            Slaughter King                                                 2   
            Sneakin’                                                       1   
            The Slaughter Tape                                             1   
            Without Warning                                                2   
            i am > i was                                                   3   
AC/DC       '74 Jailbreak                                                 13   
            12 of the Best                                                 2   
            1976-12-18: A Giant Doze of Rock 'n' Roll: Fest...             2   
            1979-10-16: Living in the Hell: Towson, MD, USA                2   
            1979-11-12: Jaap Edenhal, Amsterdam, The Nether...             1   
            1986-02-15: Scandinavium, Gothenburg, Västra Gö...             1   
            1995-01-15: Rock in Rio: Cidade do Rock, Rio de...             1   
            1996 Ballbreaker Australian Tour EP                            1   
            2003-06-22: Sympathy For The Devil: Live am 22....             1   
            2007 Gold Edition                                              1   
            3 Record Set                                                   1   
            A Vulgar Display of Ultra Rare Tracks                          1   
...                                                                      ...   
Zayn        Back to Sleep (remix)                                          1   
            BeFoUr                                                         1   
            Cruel                                                          1   
            Cruel (Jay Pryor remix)                                        1   
            Cruel (MXXWLL remix)                                           1   
            Cruel (Ta-ku remix)                                            1   
            Dusk Till Dawn                                                 1   
            Dusk Till Dawn (The Remixes)                                   1   
            Dusk Till Dawn (radio edit)                                    1   
            Entertainer                                                    1   
            Fingers                                                        1   
            Freedun                                                        1   
            I Don't Wanna Live Forever (Fifty Shades Darker)               1   
            I Don’t Wanna Live Forever (Fifty Shades Darker)               1   
            Icarus Falls                                                   5   
            LIKE I WOULD                                                   1   
            LIKE I WOULD (The Remixes)                                     1   
            LIKE I WOULD (The White Panda remix)                

In [20]:
df.dtypes

artist_name             object
title                   object
release_date    datetime64[ns]
primary_type            object
dtype: object

In [21]:
df.sort_values(by=['artist_name','title','release_date'],inplace=True)

In [22]:
df.head()

,artist_name,title,release_date,primary_type
8810,21 Savage,1Night,2018-04-27,Single
8800,21 Savage,Bartier Cardi,2017-12-22,Single
8812,21 Savage,Bartier Cardi,2017-12-22,Single
8802,21 Savage,Enzo,2016-04-24,Single
8815,21 Savage,Focus,2019-06-13,Single


In [23]:
df = df.groupby(['artist_name','title']).first()

In [24]:
df.reset_index(inplace=True)

In [25]:
df.head()

,artist_name,title,release_date,primary_type
0,21 Savage,1Night,2018-04-27,Single
1,21 Savage,Bartier Cardi,2017-12-22,Single
2,21 Savage,Enzo,2016-04-24,Single
3,21 Savage,Focus,2019-06-13,Single
4,21 Savage,Free Guwop EP,2015-07-02,Album


In [26]:
df.count()

artist_name     9416
title           9416
release_date    9416
primary_type    9356
dtype: int64

In [27]:
df.to_csv('musicbrainz_data.csv')